# 0. Imports and Setting up Anthropic API Client

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [ ]:
# Load Prompts and Problem Description
# Variables Prompt
prompt11_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt11_MathematicalModel.txt'

# Objective Prompt
prompt12_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt12_MathematicalModel.txt'

# Constraint Prompt
prompt13_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt13_MathematicalModel.txt'

# Code Prompt
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/MIP/MIP2.txt'

prompt11_file = open(prompt11_path, "r")
prompt12_file = open(prompt12_path, "r")
prompt13_file = open(prompt13_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt11 = prompt11_file.read()
print("Prompt 1.1 (Variables):\n", prompt11)

prompt12 = prompt12_file.read()
print("Prompt 1.2 (Objctive):\n", prompt12)

prompt13 = prompt13_file.read()
print("Prompt 1.3 (Constraints):\n", prompt13)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1.1 (Variables):
 Please formulate only the variables for this mathematical optimization problem. 
Prompt 1.2 (Objctive):
 Please formulate only the objective function for this mathematical optimization problem. 
Prompt 1.3 (Constraints):
 Please formulate only the constraints for this mathematical optimization problem. 
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also a

In [ ]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1.1: Create Variables for Mathematical Model

In [ ]:
message11 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt11,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p11 = message11.content[0].text

In [ ]:
# Print response
print(response_p11)

To formulate the variables for this mathematical optimization problem, let's define the following:

Indices:
- i: product index (i = 1, 2 for food; i = 3, 4, 5 for beverages)
- t: time period index (t = 1, 2, ..., 12)

Decision variables:
- x[i,t]: production quantity of product i in period t (integer for food, continuous for beverages)
- s[i,t]: storage quantity of product i at the end of period t (integer for food, continuous for beverages)
- y[i,t]: binary variable indicating whether product i is produced in period t (1 if produced, 0 otherwise)

Input parameters:
- f[i,t]: fixed cost of producing product i in period t
- cp[i,t]: variable cost of producing one unit of product i in period t
- cs[i,t]: variable cost of storing one unit of product i in period t
- d[i,t]: maximum demand for product i in period t
- r[i,t]: revenue per unit of product i sold in period t
- SC[i]: storage capacity for product i (580, 687, 599, 788, 294)
- PC[i]: production capacity for product i (1080, 908,

# 1. Prompt 1.2: Create Objective for Mathematical Model

In [ ]:
message12 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt12,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": (problem_desc + response_p11)}
    ]
)

response_p12 = message12.content[0].text

In [ ]:
# Print response
print(response_p12)

Based on the information provided, the objective function for this mathematical optimization problem can be formulated as follows:

Maximize:
12
Σ
t=1
5
Σ
i=1
(r[i,t] * min(x[i,t] + s[i,t-1], d[i,t]) - f[i,t] * y[i,t] - cp[i,t] * x[i,t] - cs[i,t] * s[i,t])

where:
- min(x[i,t] + s[i,t-1], d[i,t]) represents the actual sales of product i in period t, considering the available quantity (production + previous storage) and the maximum demand.
- r[i,t] * min(x[i,t] + s[i,t-1], d[i,t]) represents the revenue from selling product i in period t.
- f[i,t] * y[i,t] represents the fixed cost of production for product i in period t, where y[i,t] is a binary variable indicating whether production occurs.
- cp[i,t] * x[i,t] represents the variable cost of production for product i in period t.
- cs[i,t] * s[i,t] represents the variable cost of storage for product i at the end of period t.

The objective function aims to maximize the total profit over all time periods and products by subtracting the f

# 1. Prompt 1.3: Create Constraints for Mathematical Model


In [ ]:
message13 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt13,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": (problem_desc + response_p11 + response_p12)}
    ]
)

response_p13 = message13.content[0].text

In [ ]:
# Print response
print(response_p13)

Thank you for the detailed problem description. I will now formulate the constraints for this mathematical optimization problem.

Constraints:

1. Production capacity constraints:
   - For food products (i = 1, 2):
     x[i,t] ≤ PC[i] * y[i,t], for all t
   - For beverage products (i = 3, 4, 5):
     x[i,t] ≤ PC[i] * y[i,t], for all t

2. Storage capacity constraints:
   s[i,t] ≤ SC[i], for all i and t

3. Storage balance constraints:
   s[i,t] = s[i,t-1] + x[i,t] - min(x[i,t] + s[i,t-1], d[i,t]), for all i and t > 1
   s[i,1] = x[i,1] - min(x[i,1], d[i,1]), for all i

4. Non-negativity and integer constraints:
   - For food products (i = 1, 2):
     x[i,t] ≥ 0 and integer, for all t
     s[i,t] ≥ 0 and integer, for all t
   - For beverage products (i = 3, 4, 5):
     x[i,t] ≥ 0, for all t
     s[i,t] ≥ 0, for all t

5. Binary constraints:
   y[i,t] ∈ {0, 1}, for all i and t

Explanation:
1. The production capacity constraints ensure that the production quantity of each product in each

# 2. Prompt 2: Write Code based on Mathematical Model

In case more than the component of each step was generated, use "last instance"/ "newest version" of the components.

Example:
Prompt 1.1 (Variables): Model generates only variables
Prompt 1.2 (Objectvie): Model generates objective and constraints
Prompt 1.3 (Constraints): Model generates only constraints

Use Variables from step 1.1, use objective from 1.2 and use constraints from 1.3 (since constraints version 1.3 is more recent than constraints version 1.2).

This set up allows for potential self-correction of the model throughout the three prompts.

Correctness of each component is still generated in the respective step, so self-correction is independent of this.

In [ ]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": (response_p11 + response_p12 + response_p13)}
    ]
)

response_p2 = message2.content[0].text

In [ ]:
# Print response
print(response_p2)

Great! Now let's implement the mathematical optimization problem using Python and Pyomo. Here's the code:

```python
import pyomo.environ as pyo
import pandas as pd

# Read input data from CSV files (replace with actual file paths)
df_f = pd.read_csv('fixed_cost.csv')
df_cp = pd.read_csv('variable_production_cost.csv')
df_cs = pd.read_csv('variable_storage_cost.csv')
df_d = pd.read_csv('maximum_demand.csv')
df_r = pd.read_csv('unit_revenue.csv')

# Define the model
model = pyo.ConcreteModel()

# Define the sets
model.I = pyo.RangeSet(1, 5)  # Product index
model.T = pyo.RangeSet(1, 12)  # Time period index

# Define the parameters
model.f = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_f.iloc[t-1, i-1])
model.cp = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_cp.iloc[t-1, i-1])
model.cs = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_cs.iloc[t-1, i-1])
model.d = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_d.iloc[t-1, i-1

# 4. Input Problem Data and Test Model Code

In [ ]:
# Installing pyomo nd solver
!pip install -q pyomo
!pip install pandas
!apt-get install -y -qq glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.0 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121913 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [ ]:
import pyomo.environ as pyo
import pandas as pd

# Read input data from CSV files (replace with actual file paths)
fixed_cost_production = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/fixed_cost_production.csv")
df_f = fixed_cost_production.drop("Unnamed: 0", axis = 1).transpose()
df_f.columns = df_f.columns.astype(int)

variable_cost_production = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/variable_cost_production.csv")
df_cp = variable_cost_production.drop("Unnamed: 0", axis = 1).transpose()
df_cp.columns = df_cp.columns.astype(int)

variable_cost_storage = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/variable_cost_storage.csv")
df_cs = variable_cost_storage.drop("Unnamed: 0", axis = 1).transpose()
df_cs.columns = df_cs.columns.astype(int)

demand = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/demand.csv")
df_d = demand.drop("Unnamed: 0", axis = 1).transpose()
df_d.columns = df_d.columns.astype(int)

revenue = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/revenue.csv")
df_r = revenue.drop("Unnamed: 0", axis = 1).transpose()
df_r.columns = df_r.columns.astype(int)

# Define the model
model = pyo.ConcreteModel()

# Define the sets
model.I = pyo.RangeSet(1, 5)  # Product index
model.T = pyo.RangeSet(1, 12)  # Time period index

# Define the parameters
model.f = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_f.iloc[t-1, i-1])
model.cp = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_cp.iloc[t-1, i-1])
model.cs = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_cs.iloc[t-1, i-1])
model.d = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_d.iloc[t-1, i-1])
model.r = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_r.iloc[t-1, i-1])

model.SC = pyo.Param(model.I, initialize={1: 580, 2: 687, 3: 599, 4: 788, 5: 294})  # Sample data for storage capacity
model.PC = pyo.Param(model.I, initialize={1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403})  # Sample data for production capacity

# Define the variables
model.x = pyo.Var(model.I, model.T, domain=pyo.NonNegativeIntegers)
model.s = pyo.Var(model.I, model.T, domain=pyo.NonNegativeIntegers)
model.y = pyo.Var(model.I, model.T, domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(model.r[i,t] * pyo.min(model.x[i,t] + model.s[i,t-1], model.d[i,t]) -
               model.f[i,t] * model.y[i,t] - model.cp[i,t] * model.x[i,t] - model.cs[i,t] * model.s[i,t]
               for i in model.I for t in model.T)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def production_capacity_rule(model, i, t):
    return model.x[i,t] <= model.PC[i] * model.y[i,t]
model.production_capacity = pyo.Constraint(model.I, model.T, rule=production_capacity_rule)

def storage_capacity_rule(model, i, t):
    return model.s[i,t] <= model.SC[i]
model.storage_capacity = pyo.Constraint(model.I, model.T, rule=storage_capacity_rule)

def storage_balance_rule(model, i, t):
    if t == 1:
        return model.s[i,t] == model.x[i,t] - pyo.min(model.x[i,t], model.d[i,t])
    else:
        return model.s[i,t] == model.s[i,t-1] + model.x[i,t] - pyo.min(model.x[i,t] + model.s[i,t-1], model.d[i,t])
model.storage_balance = pyo.Constraint(model.I, model.T, rule=storage_balance_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print(f"Objective value: {pyo.value(model.obj):.2f}")
for i in model.I:
    for t in model.T:
        print(f"Product {i}, Period {t}: Production = {pyo.value(model.x[i,t])}, Storage = {pyo.value(model.s[i,t])}")

ERROR:pyomo.core:Rule failed when generating expression for Objective obj with index None:
AttributeError: module 'pyomo.environ' has no attribute 'min'
ERROR:pyomo.core:Constructing component 'obj' from data=None failed:
    AttributeError: module 'pyomo.environ' has no attribute 'min'


AttributeError: module 'pyomo.environ' has no attribute 'min'

# 5. Correct The Model Code to Test Mathematical Model (if applicable)

In [ ]:
# Download Gurobi
!wget https://packages.gurobi.com/9.5/gurobi9.5.2_linux64.tar.gz

# Extract the tarball
!tar -xvzf gurobi9.5.2_linux64.tar.gz

# Set up environment variables for Gurobi
import os
os.environ['GUROBI_HOME'] = "/content/gurobi952/linux64"
os.environ['PATH'] += ":/content/gurobi952/linux64/bin"
os.environ['LD_LIBRARY_PATH'] = "/content/gurobi952/linux64/lib"

--2024-06-08 10:50:50--  https://packages.gurobi.com/9.5/gurobi9.5.2_linux64.tar.gz
Resolving packages.gurobi.com (packages.gurobi.com)... 108.156.201.71, 108.156.201.49, 108.156.201.82, ...
Connecting to packages.gurobi.com (packages.gurobi.com)|108.156.201.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75338907 (72M) [application/x-gzip]
Saving to: ‘gurobi9.5.2_linux64.tar.gz’

gurobi9.5.2_linux64 100%[===================>]  71.85M  42.4MB/s    in 1.7s    

2024-06-08 10:50:53 (42.4 MB/s) - ‘gurobi9.5.2_linux64.tar.gz’ saved [75338907/75338907]

gurobi952/
gurobi952/linux64/
gurobi952/linux64/lib/
gurobi952/linux64/lib/libgurobi_g++5.2.a
gurobi952/linux64/lib/gurobi.jar
gurobi952/linux64/lib/gurobi-javadoc.jar
gurobi952/linux64/lib/gurobi95.netstandard20.dll
gurobi952/linux64/lib/gurobi95.netstandard20.xml
gurobi952/linux64/lib/libgurobi.so.9.5.2
gurobi952/linux64/lib/libgurobi95_light.so
gurobi952/linux64/lib/libGurobiJni95.so
gurobi952/linux64/lib/libg

In [ ]:
import shutil
shutil.move('/content/drive/MyDrive/gurobi.lic', '/root/gurobi.lic')

'/root/gurobi.lic'

In [ ]:
##### Linearized min functions in objective and constraints #####
#####        Added correct decision variable domains        #####

import pyomo.environ as pyo
import pandas as pd

# Read input data from CSV files (replace with actual file paths)
fixed_cost_production = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/fixed_cost_production.csv")
df_f = fixed_cost_production.drop("Unnamed: 0", axis=1).transpose()
df_f.columns = df_f.columns.astype(int)

variable_cost_production = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/variable_cost_production.csv")
df_cp = variable_cost_production.drop("Unnamed: 0", axis=1).transpose()
df_cp.columns = df_cp.columns.astype(int)

variable_cost_storage = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/variable_cost_storage.csv")
df_cs = variable_cost_storage.drop("Unnamed: 0", axis=1).transpose()
df_cs.columns = df_cs.columns.astype(int)

demand = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/demand.csv")
df_d = demand.drop("Unnamed: 0", axis=1).transpose()
df_d.columns = df_d.columns.astype(int)

revenue = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/MIP/MIP2/revenue.csv")
df_r = revenue.drop("Unnamed: 0", axis=1).transpose()
df_r.columns = df_r.columns.astype(int)

# Define the model
model = pyo.ConcreteModel()

# Define the sets
model.I = pyo.RangeSet(1, 5)  # Product index
model.T = pyo.RangeSet(1, 12)  # Time period index

# Define the parameters
model.f = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_f.iloc[t-1, i-1])
model.cp = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_cp.iloc[t-1, i-1])
model.cs = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_cs.iloc[t-1, i-1])
model.d = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_d.iloc[t-1, i-1])
model.r = pyo.Param(model.I, model.T, initialize=lambda model, i, t: df_r.iloc[t-1, i-1])

model.SC = pyo.Param(model.I, initialize={1: 580, 2: 687, 3: 599, 4: 788, 5: 294})  # Sample data for storage capacity
model.PC = pyo.Param(model.I, initialize={1: 1080, 2: 908, 3: 408, 4: 1000, 5: 403})  # Sample data for production capacity

# Define the variables
model.x = pyo.Var(model.I, model.T, domain=pyo.NonNegativeIntegers)
model.s = pyo.Var(model.I, model.T, domain=pyo.NonNegativeIntegers)
model.y = pyo.Var(model.I, model.T, domain=pyo.Binary)
model.m = pyo.Var(model.I, model.T, domain=pyo.NonNegativeIntegers)  # Auxiliary variable for min(x + s[t-1], d)

#set domain to integer for food MODIFIED ADDED INTEGRALITY HERE INSTEAD OF CONSTRAINT
for t in model.T:
    model.x[5, t].domain = pyo.NonNegativeReals
    model.x[4, t].domain = pyo.NonNegativeReals
    model.x[3, t].domain = pyo.NonNegativeReals
    model.s[5, t].domain = pyo.NonNegativeReals
    model.s[4, t].domain = pyo.NonNegativeReals
    model.s[3, t].domain = pyo.NonNegativeReals
    model.m[5, t].domain = pyo.NonNegativeReals
    model.m[4, t].domain = pyo.NonNegativeReals
    model.m[3, t].domain = pyo.NonNegativeReals

# Define the objective function
def obj_rule(model):
    return sum(model.r[i, t] * model.m[i, t] -
               model.f[i, t] * model.y[i, t] -
               model.cp[i, t] * model.x[i, t] -
               model.cs[i, t] * model.s[i, t]
               for i in model.I for t in model.T)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def production_capacity_rule(model, i, t):
    return model.x[i, t] <= model.PC[i] * model.y[i, t]
model.production_capacity = pyo.Constraint(model.I, model.T, rule=production_capacity_rule)

def storage_capacity_rule(model, i, t):
    return model.s[i, t] <= model.SC[i]
model.storage_capacity = pyo.Constraint(model.I, model.T, rule=storage_capacity_rule)

def storage_balance_rule(model, i, t):
    if t == 1:
        return model.s[i, t] == model.x[i, t] - model.m[i, t]
    else:
        return model.s[i, t] == model.s[i, t-1] + model.x[i, t] - model.m[i, t]
model.storage_balance = pyo.Constraint(model.I, model.T, rule=storage_balance_rule)

def min_demand_constraint(model, i, t):
    if t == 1:
        return model.m[i, t] <= model.x[i, t] + 0
    else:
        return model.m[i, t] <= model.x[i, t] + model.s[i, t-1]
model.min_demand_constraint = pyo.Constraint(model.I, model.T, rule=min_demand_constraint)

def min_demand_constraint_demand(model, i, t):
    return model.m[i, t] <= model.d[i, t]
model.min_demand_constraint_demand = pyo.Constraint(model.I, model.T, rule=min_demand_constraint_demand)

# Solve the model
solver = pyo.SolverFactory('gurobi')
results = solver.solve(model)

# Print the results
print(f"Objective value: {pyo.value(model.obj):.2f}")
for i in model.I:
    for t in model.T:
        print(f"Product {i}, Period {t}: Production = {pyo.value(model.x[i, t])}, Storage = {pyo.value(model.s[i, t])}")

Objective value: 4523.29
Product 1, Period 1: Production = -0.0, Storage = -0.0
Product 1, Period 2: Production = -0.0, Storage = -0.0
Product 1, Period 3: Production = -0.0, Storage = -0.0
Product 1, Period 4: Production = -0.0, Storage = -0.0
Product 1, Period 5: Production = -0.0, Storage = -0.0
Product 1, Period 6: Production = -0.0, Storage = -0.0
Product 1, Period 7: Production = -0.0, Storage = -0.0
Product 1, Period 8: Production = -0.0, Storage = -0.0
Product 1, Period 9: Production = -0.0, Storage = -0.0
Product 1, Period 10: Production = -0.0, Storage = -0.0
Product 1, Period 11: Production = -0.0, Storage = -0.0
Product 1, Period 12: Production = -0.0, Storage = -0.0
Product 2, Period 1: Production = -0.0, Storage = -0.0
Product 2, Period 2: Production = -0.0, Storage = -0.0
Product 2, Period 3: Production = -0.0, Storage = -0.0
Product 2, Period 4: Production = -0.0, Storage = -0.0
Product 2, Period 5: Production = -0.0, Storage = -0.0
Product 2, Period 6: Production = -0.